# Using Google Directions API for speed calculation

You can generate and send google directions API requests based on the network. The process will create a 'car' modal
subgraph and generate API requests for all edges in the subgraph. The number of requests is at most the number of edges
in the subgraph. The process simplifies edges using `osmnx` library's method to extract a chains of nodes with no
intersections, in this way reducing the number of requests. If your graph is already simplified, the number of requests
will be equal to the number of edges.

In [1]:
from genet import read_matsim, google_directions
import genet
from genet.utils.persistence import ensure_dir

import numpy as np
import geopandas as gpd
import json
import pandas as pd
import ast
import os
import itertools
import logging
import osmnx as ox
import polyline
from shapely.geometry import LineString

# !pip3 install momepy
import momepy

/Applications/anaconda3/envs/genet/lib/python3.7/site-packages/geopandas/_compat.py:110: UserWarning: The Shapely GEOS version (3.8.0-CAPI-1.13.1 ) is incompatible with the GEOS version PyGEOS was compiled with (3.10.0-CAPI-1.16.0). Conversions between both will be slow.
  shapely_geos_version, geos_capi_version_string


## 1. Creating requests

First of all, we need to read in the network for which the requests are being generated

In [49]:
path_to_matsim_network = '../example_data/pt2matsim_network'

network = os.path.join(path_to_matsim_network, 'network.xml')
schedule = os.path.join(path_to_matsim_network, 'schedule.xml')
vehicles = os.path.join(path_to_matsim_network, 'vehicles.xml')

n = read_matsim(
    path_to_network=network, 
    epsg='epsg:27700', 
    path_to_schedule=schedule, 
    path_to_vehicles=vehicles
)

# you don't need to read the vehicles file, but doing so ensures all vehicles in the schedule 
# are of the expected type and the definition of the vehicle is preserved
n.print()

Graph info: Name: Network graph
Type: MultiDiGraph
Number of nodes: 1662
Number of edges: 3166
Average in degree:   1.9049
Average out degree:   1.9049 
Schedule info: Schedule:
Number of services: 9
Number of routes: 68
Number of stops: 45


Next we define the function to generate requests; it takes as input the network and the list of osm tags

In [3]:
# osm_tags = takes a list of osm tags to filter the network on, e.g. ['primary', 'secondary', 'tertiary']

def generate_requests(n, osm_tags):
    subgraph = n.subgraph_on_link_conditions(
            conditions = [
                {'attributes': {'osm:way:highway': {'text': osm_tags}}},
                {'modes' : 'car'}],
            how = all, 
            mixed_dtypes = True)

    simple_paths = list(ox.simplification._get_paths_to_simplify(subgraph))
    node_diff = set(subgraph.nodes) - set(itertools.chain.from_iterable(simple_paths))
    non_simplified_edges = set(subgraph.out_edges(node_diff)) | set(subgraph.in_edges(node_diff))
    all_paths = list(non_simplified_edges) + simple_paths

    api_requests = {}

    for path in all_paths:
        request_nodes = (path[0], path[-1])
        api_requests[request_nodes] = {
            'path_nodes': path,
            'path_polyline': polyline.encode([(n.node(node)['lat'], n.node(node)['lon']) for node in path]),
            'origin': n.node(request_nodes[0]),
            'destination': n.node(request_nodes[1])}

    return api_requests

Let's create requests to get speed information for the main roads, print the number of requests, and save requests in a json file.

In [16]:
primary_tags = ['motorway', 'trunk', 'motorway,trunk', 'M50', 'primary']
requests = generate_requests(n, primary_tags)
print(len(requests))

genet.utils.google_directions.dump_all_api_requests_to_json(requests, '../example_data/', 'api_requests_send.json')

2021-11-23 10:05:34,682 - Saving Google Directions API requests to ../example_data/


136


136 requests were generated. The number of requests to be sent is important as it will influence the cost of Google Directions API. Current pricing can be found here: https://developers.google.com/maps/documentation/directions/usage-and-billing

It can be useful to visualise the requests before sending them, to confirm that they are as expected. Section 5 explains how to visualise requests you generate and results you receive from the API using Kepler. 

## 2. Sending requests

### 2.1 Sending requests created in Section 1

To send requests to Google Direction API you need a key [(read more here)](https://developers.google.com/maps/documentation/directions/start). After obtaining a key, you can either pass it to the elevant function directly

In [9]:
# Specify your own key
api_key = 'YOUR_API_KEY'

In [17]:
# Read in the API requests generated in the previous section
path = '../example_data/api_requests_send.json'
api_requests = genet.utils.google_directions.read_api_requests(path)

Before we send the requests, we need to specify the time of departure:
- `None` to get duration based on road network and average time-independent traffic conditions
- `now` for current time
- Time in the future as an integer in seconds since midnight, January 1, 1970 UTC, i.e. unix time (this website is useful for converting to unix time https://www.epochconverter.com )

More details about `departure_time` parameter can be found here: https://developers.google.com/maps/documentation/directions/get-directions#departure_time

If you set `departure_time` parameter to `now` or a time in the future, you can also specify the `traffic_model` parameter:
- `best_guess` (default): indicates that the returned duration_in_traffic should be the best estimate of travel time given what is known about both historical traffic conditions and live traffic. Live traffic becomes more important the closer the departure_time is to now.
- `pessimistic`: indicates that the returned duration_in_traffic should be longer than the actual travel time on most days, though occasional days with particularly bad traffic conditions may exceed this value.
- `optimistic`: indicates that the returned duration_in_traffic should be shorter than the actual travel time on most days, though occasional days with particularly good traffic conditions may be faster than this value.

More details about `traffic_model` parameter can be found here: https://developers.google.com/maps/documentation/directions/get-directions#traffic_model

In [18]:
# You may want to set a limit on the number of requests you are happy to send, to avoid excess costs if there is a mistake
lim = 136

if len(api_requests) < lim:
    output_dir = '../example_data/example_google_speed_data/'
    filename = 'api_requests.geojson'

    logging.info('Sending API requests')
    api_requests = genet.google_directions.send_requests(api_requests=api_requests,
                                                         departure_time=1665563400,
                                                         traffic_model='best_guess',
                                                         key=api_key)      

    logging.info('Parsing API requests')
    api_requests = genet.google_directions.parse_results(api_requests=api_requests)

    logging.info('Saving API requests')
    genet.google_directions.dump_all_api_requests_to_json(api_requests=api_requests, 
                                                          output_dir=output_dir, 
                                                          output_file_name=filename)

This method will save derived results in the output directory provided, an example can be found here: 
`../example_data/example_google_speed_data`. 

It comprises of the google polyline of the response and speed derived from distance and time taken to travel as well as information that was generated in order to make the response such as the node IDs in the network for which this response holds, the `path_nodes` which denote any extra nodes from the non-simplified chain of nodes/edges in the request, the polyline of the network path, encoded using the same polyline encoding as the Google request polyline; as well as spatial information about the origin and destination of the request and timestamp.

### 2.2 Generating and sending requests in one step

The steps described above give you flexibility in choosing the parts of the network for which you want to send the requests. If you are happy to send the requests for the whole network, you can skip sections 1 and 2.1 and go straight to the steps below (you still need to load the network first).

If you know the API key, you can specify it in the function call:

Or set it as an environmental variable called `GOOGLE_DIR_API_KEY`, if using command line: `$ export GOOGLE_DIR_API_KEY='key'`

If you use AWS, you can also store the key in the `Secrets Manager` [(read more here)](https://aws.amazon.com/secrets-manager/)
authenticate to your AWS account and then pass the `secret_name` and `region_name` to the `send_requests_for_network` 
method:

## 3. Processing the requests

### 3.1 Attaching the speed values from requests to the network

Once the request results have been received, you can read them in from the output directory you specified. 

In [50]:
api_requests = google_directions.read_api_requests('../example_data/example_google_speed_data/api_requests.json')

In [51]:
api_requests

{('9791490', '4698712638'): {'path_nodes': ('9791490', '4698712638'),
  'path_polyline': 'mvmyHpqYb@lA',
  'origin': {'id': '9791490',
   'x': 529414.5591563961,
   'y': 181898.4902840198,
   'lat': 51.5211862,
   'lon': -0.1360879,
   's2_id': 5221390682074967291},
  'destination': {'id': '4698712638',
   'x': 529387.9166476472,
   'y': 181877.74867097137,
   'lat': 51.5210059,
   'lon': -0.1364793,
   's2_id': 5221390682013665023},
  'timestamp': 1594385229.635254,
  'parsed_response': {'google_speed': 6.8, 'google_polyline': 'mvmyHpqYb@pA'}}}

Once you have results, you can attach them to the network. This will create a dictionary of non-simplified edges to which the response data applies.

In [27]:
google_edge_data = google_directions.map_results_to_edges(api_requests)

In [28]:
google_edge_data

{('9791490', '4698712638'): {'google_speed': 6.8,
  'google_polyline': 'mvmyHpqYb@pA'}}

In [29]:
n.edge('9791490', '4698712638')

{0: {'id': '596',
  'from': '9791490',
  'to': '4698712638',
  'freespeed': 4.166666666666667,
  'capacity': 600.0,
  'permlanes': 1.0,
  'oneway': '1',
  'modes': {'car'},
  's2_from': 5221390682074967269,
  's2_to': 5221390682013665025,
  'attributes': {'osm:way:access': {'name': 'osm:way:access',
    'class': 'java.lang.String',
    'text': 'no'},
   'osm:way:highway': {'name': 'osm:way:highway',
    'class': 'java.lang.String',
    'text': 'unclassified'},
   'osm:way:id': {'name': 'osm:way:id',
    'class': 'java.lang.Long',
    'text': '476247613'},
   'osm:way:name': {'name': 'osm:way:name',
    'class': 'java.lang.String',
    'text': 'Chitty Street'}},
  'length': 33.76444553419279}}

If we're working with a network that may have multiple edges between the same pair of nodes, we can restrict the links to which the data will be applied by specifying a modal condition, so that at least only links allowing cars will inherit this data.

In [30]:
def modal_condition(value):
    return 'car' in value

In [31]:
n.apply_attributes_to_edges(google_edge_data, conditions={'modes': modal_condition})

2021-11-23 10:31:48,717 - Changed Edge attributes for 1 edges


This will result in two new data points in the relevant links: `google_speed` and `google_polyline`.

In [32]:
n.edge('9791490', '4698712638')

{0: {'id': '596',
  'from': '9791490',
  'to': '4698712638',
  'freespeed': 4.166666666666667,
  'capacity': 600.0,
  'permlanes': 1.0,
  'oneway': '1',
  'modes': {'car'},
  's2_from': 5221390682074967269,
  's2_to': 5221390682013665025,
  'attributes': {'osm:way:access': {'name': 'osm:way:access',
    'class': 'java.lang.String',
    'text': 'no'},
   'osm:way:highway': {'name': 'osm:way:highway',
    'class': 'java.lang.String',
    'text': 'unclassified'},
   'osm:way:id': {'name': 'osm:way:id',
    'class': 'java.lang.Long',
    'text': '476247613'},
   'osm:way:name': {'name': 'osm:way:name',
    'class': 'java.lang.String',
    'text': 'Chitty Street'}},
  'length': 33.76444553419279,
  'google_speed': 6.8,
  'google_polyline': 'mvmyHpqYb@pA'}}

In [33]:
def speed_difference(link_attribs):
    return link_attribs['freespeed'] - link_attribs['google_speed']

In [34]:
n.apply_function_to_links(speed_difference, 'speed_difference')

2021-11-23 10:31:55,908 - 3165 out of 3166 links have not been affected by the function. Links affected: ['596']
2021-11-23 10:31:55,910 - Changed Link attributes for 1 links


You can also choose to set google speed as the `freespeed` in the network. But be mindful if you use it for MATSim
simulations, `freespeed` denotes the maximum speed a vehicle can travel on a certain link, Google Directions API data
with `departure_time='now'` should be ran late at night/early morning ~4am local time to the network for any reliable results.
Otherwise you are adding traffic conditions to the network which should be simulated by demand (population) side of 
the model rather than supply (network).

In [35]:
def set_google_speed(link_attribs):
    if link_attribs['google_speed'] != 0:
        return link_attribs['google_speed']
    else:
        return link_attribs['freespeed']

In [36]:
n.apply_function_to_links(set_google_speed, 'freespeed')

2021-11-23 10:32:00,453 - 3165 out of 3166 links have not been affected by the function. Links affected: ['596']
2021-11-23 10:32:00,456 - Changed Link attributes for 1 links


In [37]:
n.edge('9791490', '4698712638')

{0: {'id': '596',
  'from': '9791490',
  'to': '4698712638',
  'freespeed': 6.8,
  'capacity': 600.0,
  'permlanes': 1.0,
  'oneway': '1',
  'modes': {'car'},
  's2_from': 5221390682074967269,
  's2_to': 5221390682013665025,
  'attributes': {'osm:way:access': {'name': 'osm:way:access',
    'class': 'java.lang.String',
    'text': 'no'},
   'osm:way:highway': {'name': 'osm:way:highway',
    'class': 'java.lang.String',
    'text': 'unclassified'},
   'osm:way:id': {'name': 'osm:way:id',
    'class': 'java.lang.Long',
    'text': '476247613'},
   'osm:way:name': {'name': 'osm:way:name',
    'class': 'java.lang.String',
    'text': 'Chitty Street'}},
  'length': 33.76444553419279,
  'google_speed': 6.8,
  'google_polyline': 'mvmyHpqYb@pA',
  'speed_difference': -2.633333333333333}}

### 3.2 Validating google speed values

Once you have attached the google speed values to the network, you may want to do some validation, to check if the values make sense and if there are any missing values. 

To do that, we first need to convert the network to a geodataframe.

## 4. Visualising requests